<a href="https://colab.research.google.com/github/Aelamin19/Applied-predictive-analytics/blob/main/IMDB_Review_Sentiment_Prediction_%2B_Experiment_with_Transformers_Ammar_Elamin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Training data:**

Let's read our training data. Here, we have the text and label fields. The Label is 1 for positive reviews and 0 for negative reviews.

In [4]:
import pandas as pd

df1 = pd.read_csv('https://raw.githubusercontent.com/aws-samples/aws-machine-learning-university-accelerated-nlp/master/data/final_project/imdb_train.csv', header=0)

print(df1.shape)
df1.head()

(25000, 2)


,text,label
0,This movie makes me want to throw up every tim...,0
1,Listening to the director's commentary confirm...,0
2,One of the best Tarzan films is also one of it...,1
3,Valentine is now one of my favorite slasher fi...,1
4,No mention if Ann Rivers Siddons adapted the m...,0


**Test data:**

In [5]:
import pandas as pd

df2 = pd.read_csv('https://raw.githubusercontent.com/aws-samples/aws-machine-learning-university-accelerated-nlp/master/data/final_project/imdb_test.csv', header=0)

print(df2.shape)
df2.head()

(25000, 2)


,text,label
0,What I hoped for (or even expected) was the we...,0
1,Garden State must rate amongst the most contri...,0
2,There is a lot wrong with this film. I will no...,1
3,"To qualify my use of ""realistic"" in the summar...",1
4,Dirty War is absolutely one of the best politi...,1


Above i see that both the given train and test datasets have an equal number of elements and both have the same types of columns so i renamed them.

therfore i will split the datasets into training and validation

Labeling the columns of the dataframe:



In [6]:
import pandas as pd

df1 = df1.rename(columns={'text': 'review', 'label': 'positive'})
df1.head()

,review,positive
0,This movie makes me want to throw up every tim...,0
1,Listening to the director's commentary confirm...,0
2,One of the best Tarzan films is also one of it...,1
3,Valentine is now one of my favorite slasher fi...,1
4,No mention if Ann Rivers Siddons adapted the m...,0


In [7]:
import pandas as pd

df2 = df2.rename(columns={'text': 'review', 'label': 'positive'})
df2.head()

,review,positive
0,What I hoped for (or even expected) was the we...,0
1,Garden State must rate amongst the most contri...,0
2,There is a lot wrong with this film. I will no...,1
3,"To qualify my use of ""realistic"" in the summar...",1
4,Dirty War is absolutely one of the best politi...,1


Exploratory data analysis

In [8]:
df1['positive'].value_counts()

df2['positive'].value_counts()

0    12500
1    12500
Name: positive, dtype: int64

missing values:

In [9]:
print(df1.isna().sum())
print(df2.isna().sum())

review      0
positive    0
dtype: int64
review      0
positive    0
dtype: int64


There are no missing values in the dataset

Removal of stop words and stemming:

In [10]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

stop = stopwords.words('english')

excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't",
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren',
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]

stop_words = [word for word in stop if word not in excluding]

snow = SnowballStemmer('english')


In [12]:
def process_text(texts):
    final_text_list=[]
    for sent in texts:

        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ""

        filtered_sentence=[]

        sent = sent.lower() # Lowercase
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:

        for w in word_tokenize(sent):
            # We are applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stop words
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) #final string of cleaned words

        final_text_list.append(final_string)

    return final_text_list

Train - validation split

In [13]:
#I will split the data 90(training) - 10(test)
from sklearn.model_selection import train_test_split

X=df1[["review"]]
Y=df1["positive"]
X_train, X_val, y_train, y_val = train_test_split(X,
                                                  Y,
                                                  test_size=0.10,
                                                  shuffle=True,
                                                  random_state=324
                                                 )

In [14]:
print("Processing the review fields")
train_text_list = process_text(X_train["review"].tolist())
val_text_list = process_text(X_val["review"].tolist())

Processing the review fields


Pipeline - Data processing

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


import gensim
from gensim.models import Word2Vec

w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    ('text_vect', CountVectorizer(binary=True,
    #( 'text_vect', TfidfVectorizer(use_idf=True,
                                  max_features=10)),
    ('knn', KNeighborsClassifier(n_neighbors = 158))#~=square root of 25000
                                ])


# Visualize the pipeline
#from sklearn import set_config
#set_config(display='diagram')
#pipeline

Training the classifier:

In [16]:
# We use lists of processed text fields
X_train = train_text_list
X_val = val_text_list

# Fitting the Pipeline to the training data
pipeline.fit(X_train, y_train.values)

Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=10)),
                ('knn', KNeighborsClassifier(n_neighbors=158))])

Making the predictions

Testing the classifier

In [17]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)

print(val_predictions)

print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
#print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[0 1 0 ... 1 0 1]
[[741 539]
 [571 649]]
              precision    recall  f1-score   support

           0       0.56      0.58      0.57      1280
           1       0.55      0.53      0.54      1220

    accuracy                           0.56      2500
   macro avg       0.56      0.56      0.56      2500
weighted avg       0.56      0.56      0.56      2500



For the K value in the KNN, I began with the aproximate sqaure root of 25000 (the No. of elements in the dataset), 158.
then i tried some values upwards and downwards from that and they brought the f1 score further from 1 so i chose 158 which is what i judged to be the best resulting K value for the KNN

In [18]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install transformers torch scikit-learn

Retokenize the train and test data using the BertTokenixer

In [20]:
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [21]:
# Load BERT tokenizer and model, move to GPU
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)

Taking a sample of the dataset for efficiency:

In [55]:
updated_d2 = df2.iloc[24100:]
print(updated_d2.shape)

updated_d1 = df1.iloc[24100:]
print(updated_d2.shape)

(900, 2)
(900, 2)


Train - validation split (with the sample of the dataset)

In [56]:
from sklearn.model_selection import train_test_split

X=updated_d1[["review"]]
Y=updated_d1["positive"]
X_train1, X_val1, train_labels, test_labels = train_test_split(X,
                                                  Y,
                                                  test_size=0.10,
                                                  random_state=42
                                                 )

In [57]:
# Tokenize and encode the text data, move to GPU
max_length = 128



X_train_tokens = tokenizer(list(X_train1), truncation=True, padding=True, max_length=max_length, return_tensors="pt", add_special_tokens=True).to(device)
X_test_tokens = tokenizer(list(X_val1), truncation=True, padding=True, max_length=max_length, return_tensors="pt", add_special_tokens=True).to(device)


In [58]:
#Function for calculation
def get_bert_embeddings(tokens):
    embeddings = []
    for i in tqdm(range(len(tokens['input_ids']))):
        with torch.no_grad():
            output = bert_model(input_ids=tokens['input_ids'][i].unsqueeze(0), attention_mask=tokens['attention_mask'][i].unsqueeze(0))
        embeddings.append(output[0].squeeze().mean(dim=0).cpu().numpy())
    return embeddings

# Calculating BERT embeddings for the text data
X_train_bert_embeddings = get_bert_embeddings(X_train_tokens)
X_test_bert_embeddings = get_bert_embeddings(X_test_tokens)

100%|██████████| 1/1 [00:00<00:00,  7.08it/s]


We now have BERT embeddings for the text data so we can train a KNN model using scikit-learn.

In [59]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_bert_embeddings, train_labels)

NB = GaussianNB()
NB.fit(X_train_bert_embeddings, train_labels)

# Train a classifier on BERT embeddings (you can use any classifier of your choice)
# Here, we'll use Logistic Regression as an example
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_bert_embeddings, train_labels)

ValueError: ignored

Testing the classifier

In [60]:
from sklearn.metrics import classification_report, confusion_matrix

# Make predictions on the test data
predictions = knn.predict(X_test_bert_embeddings)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f'KNN Accuracy: {accuracy * 100:.2f}%')
#predictions = rf.predict(test_embeddings)
predictions = NB.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'Naive Bais Accuracy: {accuracy * 100:.2f}%')

#predictions = rf.predict(test_embeddings)
predictions = lr.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'Logistic Regression Accuracy: {accuracy * 100:.2f}%')

NotFittedError: ignored